## Prepare Dependencies

In [1]:
!pip install -U langchain-openai
!pip install openai
!pip install tiktoken
!pip install -U qdrant-client
!pip install langchain_experimental
!pip install "langchain[docarray]"

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
from langchain.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant
from langchain.chat_models import ChatOpenAI

In [3]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [53]:
from openai import OpenAI
import httpx
#Set the langchain
import os
os.environ["OPENAI_API_BASE"] = 'https://api.xty.app/v1'
os.environ["OPENAI_API_KEY"] = 'sk-x'

## Data Preparing

In [5]:
#Prepare the text of Peter Pan and load it
txt_file_path = 'peter_pan.txt'
loader = TextLoader(file_path=txt_file_path, encoding="utf-8")
doc = loader.load()

In [6]:
#Chunk the text into pieces of "chunks", or "nodes" in the llama index
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(doc)
print('done.')

done.


In [7]:
# Take a look of your documents
print (f'Now you have {len(chunks)} documents')
print(chunks[5].page_content)

Now you have 374 documents
Mr. Darling used to boast to Wendy that her mother not only loved him
but respected him. He was one of those deep ones who know about stocks
and shares. Of course no one really knows, but he quite seemed to know,
and he often said stocks were up and shares were down in a way that
would have made any woman respect him.

Mrs. Darling was married in white, and at first she kept the books
perfectly, almost gleefully, as if it were a game, not so much as a
Brussels sprout was missing; but by and by whole cauliflowers dropped
out, and instead of them there were pictures of babies without faces.
She drew them when she should have been totting up. They were Mrs.
Darling’s guesses.

Wendy came first, then John, then Michael.


In [8]:
#Set an embedding model 
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

In [9]:
#Build a vector store 
story = Qdrant.from_documents(
    chunks,
    embeddings,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="my_documents",
)

In [10]:
#Build a retriever
retriever = story.as_retriever(search_type="mmr") # Maximal marginal relevance 

## Base line of GPT-4

In [11]:
# Define a LLM
llm = ChatOpenAI(model_name="gpt-4", temperature=0, max_tokens = 150)

# Define prompt template
template = """Based on the context to answer a claim is TRUE or False, and give me your reasons. 
Make your answer concise. 
Claim: {question} 
Context: {context} 
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

/environment/miniconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [12]:
# Setup a RAG Chain
rag_chain_b = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

In [13]:
# Prepare the RAGAS for evaluation
!pip install datasets
from datasets import Dataset
!pip install ragas 
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_correctness
)
# Set the metrics of RAGAS
metrics = [
    faithfulness,
    context_precision,
    context_recall,
    answer_relevancy,
    answer_correctness
]

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


/environment/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 137.7 MB/s eta 0:00:00


In [14]:
# Prepare the queries and ground-truths
import pandas as pd
q = pd.read_csv('02-QC.csv')
q

,question,ground_truths
0,Wendy Darling met Peter Pan in the market for ...,"[""False. Wendy and Peter didn't meet in the ma..."
1,Wendy like Peter but Peter don't like her.,"[""False. Wendy like Peter indeed, Peter is als..."
2,Peter Pan has good parents.,"[""False. Peter's parents aren't mentioned. He ..."
3,Mr Darling changed after the chilren left. He ...,"[""True. He changes quite a lot after the child..."
4,Peter Pan went to Neverland through flight in ...,"[""True. Peter Pan gets to Neverland through fl..."
5,Tinker Bell drunk the poison but she came back...,"[""True. Jane mourns the death of the tinker fa..."


In [15]:
# Use the naive RAG to detect and generate answers
questions = q['question']
ground_truths = q['ground_truths']
contexts = []
answers = []

# Inference
for query in questions:
    answers.append(rag_chain_b.invoke(query))
    docs = retriever.get_relevant_documents(query)
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])
    #contexts.append(docs[0].page_content)

# To dict
data = {
    "question": questions,
    "answer": answers,
    'contexts': contexts,
    "ground_truth": ground_truths
}

In [16]:
# Evaluation of the results 
dataset = Dataset.from_dict(data)

result = evaluate(
    dataset = dataset, 
    metrics=metrics,
)

res_b = result.to_pandas()

Evaluating: 100%|██████████| 30/30 [00:43<00:00,  1.45s/it]


In [17]:
pd.set_option('max_colwidth', 700)
res_b

,question,answer,contexts,ground_truth,faithfulness,context_precision,context_recall,answer_relevancy,answer_correctness
0,Wendy Darling met Peter Pan in the market for the first time.,"False.\n\nWendy Darling did not meet Peter Pan in the market for the first time. The text describes Wendy encountering Peter Pan at her home, specifically in her bedroom when she is startled by his presence. There is a detailed description of Peter Pan's appearance and his interaction with Wendy and Mrs. Darling, but there is no mention of a market. The meeting is characterized by a personal, direct encounter in a private setting, not a public place like a market.","[She started up with a cry, and saw the boy, and somehow she knew at\nonce that he was Peter Pan. If you or I or Wendy had been there we\nshould have seen that he was very like Mrs. Darling’s kiss. He was a\nlovely boy, clad in skeleton leaves and the juices that ooze out of\ntrees but the most entrancing thing about him was that he had all his\nfirst teeth. When he saw she was a grown-up, he gnashed the little\npearls at her.\n\n\n\n\nChapter II.\nTHE SHADOW\n\n\nMrs. Darling screamed, and, as if in answer to a bell, the door opened,\nand Nana entered, returned from her evening out. She growled and sprang\nat the boy, who leapt lightly through the window. Again Mrs. Darling\nscreamed, t...","[""False. Wendy and Peter didn't meet in the market. Wendy Darling met Peter Pan when he flied in her family's window to retrieve his lost shadow for the fisrt time. ""]",NaN,0.416667,1.0,0.959023,0.233676
1,Wendy like Peter but Peter don't like her.,"TRUE.\n\nIn the provided context, Wendy expresses clear emotional sentiment towards Peter. She calls Peter's thoughts ""perfectly sweet"" and offers him a kiss, indicating affection. Moreover, she queries Peter about discussing a ""sweet subject"" with her parents, suggesting deeper feelings.\n\nConversely, Peter's responses to Wendy are detached and devoid of romantic interest. He describes his feelings as those of ""a devoted son,"" rather than a lover. This is further reinforced by his disinterest in her affectionate advances and his consistent resistance to engaging with the topic of a deeper, possibly romantic, relationship. Peter's focus on not growing up or being made a man, particularl...","[“Not if you don’t wish it,” she replied; and she distinctly heard his\nsigh of relief. “Peter,” she asked, trying to speak firmly, “what are\nyour exact feelings to me?”\n\n“Those of a devoted son, Wendy.”\n\n“I thought so,” she said, and went and sat by herself at the extreme\nend of the room.\n\n“You are so queer,” he said, frankly puzzled, “and Tiger Lily is just\nthe same. There is something she wants to be to me, but she says it is\nnot my mother.”\n\n“No, indeed, it is not,” Wendy replied with frightful emphasis. Now we\nknow why she was prejudiced against the redskins.\n\n“Then what is it?”\n\n“It isn’t for a lady to tell.”\n\n“Oh, very well,” Peter said, a little nettled. “Perha...","[""False. Wendy like Peter indeed, Peter is also very fond of Wendy because of the stories she tells, which are mainly about Peter and his adventures. He is fascinated by Wendy's stories of him that he would often visit her to listen.""]",0.666667,0.805556,0.0,0.756646,0.220213
2,Peter Pan has good parents.,"FALSE.\n\nBased on the provided context, Peter Pan does not have defined parents in typical terms within the story of ""Peter Pan."" The narrative revolves around Peter Pan's eternal childhood in Neverland, a place where he lives without traditional parents. Instead, the story mentions characters like the Darlings and Captain Hook, but these figures are not Peter's parents. The notion of ""good parents"" does not apply directly to Peter himself. Instead, discussions of parenthood within the text relate to other characters, such as the example of Mrs. Darling's care and concern shown in the excerpts, but she is not Peter Pan’s mother. Thus, saying Peter Pan has 

In [18]:
res_b.describe()

,faithfulness,context_precision,context_recall,answer_relevancy,answer_correctness
count,3.000000,6.000000,6.000000,6.000000,6.000000
mean,0.888889,0.870370,0.666667,0.744172,0.397404
std,0.192450,0.235484,0.516398,0.371487,0.307449
min,0.666667,0.416667,0.000000,0.000000,0.217347
25%,0.833333,0.854167,0.250000,0.793596,0.219607
50%,1.000000,1.000000,1.000000,0.904482,0.226944
75%,1.000000,1.000000,1.000000,0.931427,0.445897
max,1.000000,1.000000,1.000000,0.959023,0.977147


## Merger Retriever for RAG

In [19]:
#Prepare different embedding model for LOTR. 
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import SentenceTransformerEmbeddings
!pip install sentence-transformers

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [20]:
h_embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embedding = h_embedding

story_h = Qdrant.from_documents(
    chunks,
    embedding,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="my_documents",
)

In [22]:
# Import the libries for Merger Retriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers.merger_retriever import MergerRetriever

In [23]:
#Build the LOTR to avoid 'Lost in the Middle'.
retriever = story.as_retriever(search_type="mmr")
retriever_h = story_h.as_retriever(search_type="mmr")

In [24]:
#Combine the two retrievers
lotr_full = MergerRetriever(retrievers=[retriever,retriever_h])

In [25]:
#Test the Combined Retrievers.
docs = lotr_full.get_relevant_documents(
    "Is Peter Pan a brave person?")
print(len(docs))

8


In [26]:
# Avoid the futuer warnings
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [27]:
# Build the Clustering Filter
# Please pay attention to the cost

from langchain.document_transformers import EmbeddingsClusteringFilter

# This filter will divide the documents vectors into clusters or "centers" of meaning.
# Then it will pick the closest document to that center for the final results.
# By default the result document will be ordered/grouped by clusters.
filter_ordered_cluster = EmbeddingsClusteringFilter(
    embeddings=embeddings,
    num_clusters=5,
    num_closest=1,
)

# If you want the final document to be ordered by the original retriever scores
# you need to add the "sorted" parameter.(sorted=True)

pipeline = DocumentCompressorPipeline(transformers=[filter_ordered_cluster])
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline, base_retriever=lotr_full
)

docs = compression_retriever.get_relevant_documents(
    "Is Peter Pan a brave person?")
print(len(docs))

5


## Merger Retriever 

In [29]:
retriever_f = lotr_full

In [30]:
# Setup a RAG Chain
rag_chain = (
    {"context": retriever_f, "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

In [32]:
questions = q['question']
ground_truths = q['ground_truths']
contexts = []
answers = []

# Inference
for query in questions:
    answers.append(rag_chain.invoke(query))
    docs = retriever_f.get_relevant_documents(query)
    contexts.append([docs.page_content for docs in retriever_f.get_relevant_documents(query)])
    #contexts.append(docs[0].page_content)

# To dict
data = {
    "question": questions,
    "answer": answers,
    'contexts': contexts,
    "ground_truth": ground_truths
}

In [33]:
# Evaluation of the results 
dataset = Dataset.from_dict(data)

result = evaluate(
    dataset = dataset, 
    metrics=metrics,
)

res_f = result.to_pandas()

Evaluating: 100%|██████████| 30/30 [00:46<00:00,  1.54s/it]


In [34]:
res_f

,question,answer,contexts,ground_truth,faithfulness,context_precision,context_recall,answer_relevancy,answer_correctness
0,Wendy Darling met Peter Pan in the market for the first time.,"FALSE.\n\nWendy Darling first encountered Peter Pan not in the market, but in her own bedroom. The provided text describes Mrs. Darling’s interaction with Peter, intimating that Wendy, too, would have recognized him instantly there. This interaction occurs at the Darling household, as indicated by the scene of Peter escaping through the window and Mrs. Darling running out into the street out of concern for him.","[She started up with a cry, and saw the boy, and somehow she knew at\nonce that he was Peter Pan. If you or I or Wendy had been there we\nshould have seen that he was very like Mrs. Darling’s kiss. He was a\nlovely boy, clad in skeleton leaves and the juices that ooze out of\ntrees but the most entrancing thing about him was that he had all his\nfirst teeth. When he saw she was a grown-up, he gnashed the little\npearls at her.\n\n\n\n\nChapter II.\nTHE SHADOW\n\n\nMrs. Darling screamed, and, as if in answer to a bell, the door opened,\nand Nana entered, returned from her evening out. She growled and sprang\nat the boy, who leapt lightly through the window. Again Mrs. Darling\nscreamed, t...","[""False. Wendy and Peter didn't meet in the market. Wendy Darling met Peter Pan when he flied in her family's window to retrieve his lost shadow for the fisrt time. ""]",NaN,0.607143,0.000000,0.958771,0.230734
1,Wendy like Peter but Peter don't like her.,"FALSE.\n\nReasons:\n\n1. Peter acknowledges Wendy by describing his feelings as those of a ""devoted son,"" which suggests affection but not romantic interest (first document).\n2. Wendy seems to have romantic feelings for Peter, as evidenced by her willingness to give him a kiss and her affected behavior upon realizing he doesn't understand what a kiss is (third document).\n3. Peter's interactions with Wendy, including dismissing the idea of speaking to her parents about ""a very sweet subject"" and his reluctance to grow up or be caught by anyone, show a lack of reciprocal romantic interest (seventh and eighth documents).\n4. Peter's continued detachment from adult concepts and responsibil...","[“Not if you don’t wish it,” she replied; and she distinctly heard his\nsigh of relief. “Peter,” she asked, trying to speak firmly, “what are\nyour exact feelings to me?”\n\n“Those of a devoted son, Wendy.”\n\n“I thought so,” she said, and went and sat by herself at the extreme\nend of the room.\n\n“You are so queer,” he said, frankly puzzled, “and Tiger Lily is just\nthe same. There is something she wants to be to me, but she says it is\nnot my mother.”\n\n“No, indeed, it is not,” Wendy replied with frightful emphasis. Now we\nknow why she was prejudiced against the redskins.\n\n“Then what is it?”\n\n“It isn’t for a lady to tell.”\n\n“Oh, very well,” Peter said, a little nettled. “Perha...","[""False. Wendy like Peter indeed, Peter is also very fond of Wendy because of the stories she tells, which are mainly about Peter and his adventures. He is fascinated by Wendy's stories of him that he would often visit her to listen.""]",NaN,1.000000,0.000000,0.903483,0.221073
2,Peter Pan has good parents.,"FALSE.\n\nPeter Pan's situation does not involve traditionally good parental figures as conventionally understood. In J.M. Barrie's narrative, Peter Pan is depicted as an eternal child living in Neverland, devoid of direct parental care and guidance, which contrasts with conventional expectations of ""good"" parenting that involves nurturing and overseeing the development of children. Peter's existence in a fantastical realm with fairies and adventures, separate from any adult supervision or conventional family structure, underscores the absence of parental presence, traditionally essential for deeming parenting as ""good.""","[“Yes, he is rather cocky,” Wendy admitted with regret. Her mother had\nbeen question

In [35]:
res_f.describe()

,faithfulness,context_precision,context_recall,answer_relevancy,answer_correctness
count,3.000000,6.000000,6.000000,6.000000,6.000000
mean,0.583333,0.865873,0.388889,0.770122,0.554974
std,0.520416,0.166552,0.490653,0.377792,0.378003
min,0.000000,0.607143,0.000000,0.000000,0.211094
25%,0.375000,0.763492,0.000000,0.904862,0.223488
50%,0.750000,0.927579,0.166667,0.913834,0.473485
75%,0.875000,1.000000,0.833333,0.927774,0.904886
max,1.000000,1.000000,1.000000,0.958771,0.982936


## Merger Retriever with Filter and Reorder

In [36]:
retriever_m = compression_retriever

In [37]:
# Setup a RAG Chain
rag_chain = (
    {"context": retriever_m, "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

In [38]:
questions = q['question'].tail(1)
ground_truths = q['ground_truths'].tail(1)
contexts = []
answers = []

# Inference
for query in questions:
    answers.append(rag_chain.invoke(query))
    docs = retriever_m.get_relevant_documents(query)
    contexts.append([docs.page_content for docs in retriever_m.get_relevant_documents(query)])
    #contexts.append(docs[0].page_content)

# To dict
data = {
    "question": questions,
    "answer": answers,
    'contexts': contexts,
    "ground_truth": ground_truths
}

In [39]:
# Evaluation of the results 
dataset = Dataset.from_dict(data)

result = evaluate(
    dataset = dataset, 
    metrics=metrics,
)

res = result.to_pandas()

Evaluating: 100%|██████████| 5/5 [00:18<00:00,  3.79s/it]


In [40]:
res

,question,answer,contexts,ground_truth,faithfulness,context_precision,context_recall,answer_relevancy,answer_correctness
0,Tinker Bell drunk the poison but she came back to life.,"TRUE\n\nAlthough Tinker Bell did drink the poisoned medicine intended for Peter, her life was saved by the belief of children in fairies, as Peter directly appealed to the audience of the play, breaking the fourth wall, to clap if they believed in fairies. The clapping revived her, illustrating the thematic element of belief and its power within the narrative.","[“Why not?”\n\n“It is poisoned.”\n\n“Poisoned? Who could have poisoned it?”\n\n“Hook.”\n\n“Don’t be silly. How could Hook have got down here?”\n\nAlas, Tinker Bell could not explain this, for even she did not know the\ndark secret of Slightly’s tree. Nevertheless Hook’s words had left no\nroom for doubt. The cup was poisoned.\n\n“Besides,” said Peter, quite believing himself, “I never fell asleep.”\n\nHe raised the cup. No time for words now; time for deeds; and with one\nof her lightning movements Tink got between his lips and the draught,\nand drained it to the dregs.\n\n“Why, Tink, how dare you drink my medicine?”\n\nBut she did not answer. Already she was reeling in the air.\n\n“What...","[""True. Jane mourns the death of the tinker fairy in guilt and shame, and her belief in fairies begins to arise. Jane's newfound belief in fairies revives Tinker Bell.""]",0.5,0.866667,0.0,0.840081,0.468911


In [41]:
res.describe()

,faithfulness,context_precision,context_recall,answer_relevancy,answer_correctness
count,1.0,1.000000,1.0,1.000000,1.000000
mean,0.5,0.866667,0.0,0.840081,0.468911
std,NaN,NaN,NaN,NaN,NaN
min,0.5,0.866667,0.0,0.840081,0.468911
25%,0.5,0.866667,0.0,0.840081,0.468911
50%,0.5,0.866667,0.0,0.840081,0.468911
75%,0.5,0.866667,0.0,0.840081,0.468911
max,0.5,0.866667,0.0,0.840081,0.468911
